In [145]:
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
from tkinter import *
from PIL import ImageTk, Image
import numpy as np
import pandas as pd
from cmath import phase
from sklearn.externals import joblib 
from acconeer.exptool import configs, clients
from tkinter import filedialog as fd 

import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk

import socket
import paramiko #conda install -c anaconda paramiko

In [162]:
'''
Una función que a partir del nombre del fichero npy carge los datos, los redimensione y los devuelva. 
Al estilo de la función get_data(). Devuelve "datos_bruto". 

Los test de esta función:   comprobar que devuelve un array 2D, 
                            comprobar que la segunda dimensión tiene tamaño 291, 
                            comprobar que la matriz tiene una parte real 
                            y que tiene una parte imaginaria.
'''

def get_data(url):
    diccionario = np.load(url, allow_pickle=True).item()
    data = diccionario.get('sweep_data').get('data')
    data = data.reshape(data.shape[0],data.shape[2])
    return data

In [163]:
'''
Una función get_modulo_fase() que a partir del array 2D anterior, obtenga un array 2D, con el doble de anchura. 
Por ejemplo, si "datos_bruto" es de 300x291, "modulo_fase" será de 600x291. 
Esa función obtendrá el módulo del número complejo y la fase del número complejo. 
    Módulo será una matriz de 300x291 y fase también. 
Será necesario concatenarlas "horizontalmente". 
Los test comprobarán las dimensiones y tamaños.
'''

def get_modulo_fase(data):
    modulo = abs(data)
    fase = []
    for i in data:
        fila = []
        for j in i:
            fila.append(phase(j)) #Fase
        fase.append(fila)
    fase = np.asarray(fase)
    modulo_fase = np.concatenate((modulo, fase), axis=0)
    return np.asarray(modulo_fase)
    

In [164]:
'''
Una función que a partir de los datos "modulo_fase" devuelva la media. 
La comprobación sería que devuelva un array de 1 dimensión de tamaño 582 (291 x 2).
'''
def get_media(modulo_fase):
    traspuesta = np.transpose(modulo_fase) #Obtenemos la matriz traspuesta(291x600) de modulo_fase(600x291)
    
    #Separamos la traspuesta en modulo y fase
    t_modulo = traspuesta[:,:int(traspuesta.shape[1]/2)]
    t_fase = traspuesta[:,int(traspuesta.shape[1]/2):]
    
    media = []
    for i in t_modulo:
        media.append(i.mean())
    for j in t_fase:
        media.append(j.mean())
        
    return np.asarray(media) #Devuelve medias modulo y fase

In [171]:

def clasificar(datos):
    global label_packed
    pred = modelo.predict(datos)[0]
    salida = diccionario[pred]
    centro.configure(foreground='#011638', text=salida) 
    porcentajes = modelo.predict_proba(datos)[0]
    actualizarGrafico(porcentajes)

def mostrar_boton(datos):
    boton=Button(app,text="Clasificar material", command=lambda: clasificar(datos),padx=10,pady=5)
    boton.configure(background='#364156', foreground='white', font=('arial',10,'bold'))
    boton.place(relx=0.79,rely=0.46)

def carga_datos():
    try:
        '''
        uploaded=Image.open(file_path)
        uploaded.thumbnail(((top.winfo_width()/2.25),(top.winfo_height()/2.25)))
        im=ImageTk.PhotoImage(uploaded)
        sign_image.configure(image=im)
        sign_image.image=im
        label.configure(text='')
        '''     
        
        datos = []
        
        url = fd.askopenfilename(initialdir = "./Materiales",title = "Select file",filetypes = (("npy files","*.npy"),("all files","*.*")))
        
        data = get_data(url)
        modulofase = get_modulo_fase(data)
        media = get_media(modulofase)
        
        datos.append(media)

        centro.configure(text='')
        
        mostrar_boton(datos)
    except:
        pass
    
def carga_acconeer():
    try:
        ip = socket.gethostbyname('RadarAcconeer')

        ssh_client = paramiko.SSHClient()
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh_client.connect(ip, 22, 'pi', 'raspberry')

        #Ejecutar radar
        ssh_client.exec_command('Downloads/rpi_xc112/utils/acc_streaming_server_rpi_xc112_r2b_xr112_r2b_a111_r2c')

        #Parámetros del radar
        iq = configs.IQServiceConfig()
        envelope = configs.EnvelopeServiceConfig()
        iq.range_interval=[0.10, 0.24]
        envelope.range_interval=[0.10, 0.24]
        raspi = clients.SocketClient(ip)
        raspi.connect()
        raspi.start_session(iq)

        data = []

        for i in range(300):
            data.append(raspi.get_next()[-1])
            
        data = np.asarray(data)
        
        modulofase = get_modulo_fase(data)
        media = get_media(modulofase)

        datos = []
        datos.append(media)

        label.configure(text='')

        #ssh_client.exec_command('exit')

        mostrar_boton(datos)
    except:
        pass
    
def actualizarGrafico(porcentajes):
    global barras
    barras.clear()
    #crear dataframe
    Data1 = {'Clases': ['CARTON','CRISTAL','PLASTICO'], 'Porcentaje': porcentajes}
    df1 = pd.DataFrame(Data1, columns= ['Clases', 'Porcentaje'])
    df1 = df1[['Clases', 'Porcentaje']].groupby('Clases').sum()
    #Agregar data al grafico de barras
    df1.plot(kind='bar', legend=True, ax=barras)
    barras.set_title('Porcentaje de coincidencia')
    bar1.draw()

In [172]:
def createAboutUs():
    aboutus = tk.Toplevel(app)
    aboutus.geometry('400x200')
    aboutus.title('AboutUs')
    label2 = tk.Label(aboutus, text = "About Us")
    
    label2.pack()


In [176]:

#Cargar modelo RandomForest
modelo = joblib.load('modeloRandomForest.pkl')

#DICCIONARIO
diccionario = { 
    'carton': 'CARTON',
    'plastico': 'PLASTICO',
    'cristal': 'CRISTAL' 
}

#Inicializar GUI
app=tk.Tk()
app.geometry('800x600')
app.title('Clasificador de materiales')
app.configure(background='lavender') #CDCDCD


heading = Label(app, text="Clasificador de materiales", font=('arial',20,'bold'))
heading.configure(background='lavender',foreground='#364156')

#------- Acerca de

aboutus = Button(heading, text="About Us",command=createAboutUs)
aboutus.configure(background='#364156', foreground='white',font=('arial',10,'bold'))
aboutus.pack(side=LEFT, padx = 50, pady=10)

heading.pack(side=TOP,fill=X,expand=True)

#------- Sección Gráficos
centro=Label(app,background='lavender', font=('arial',15,'bold'))

framegraficos = Frame(bg="#949292", width="500", height="620")
framegraficos.pack(side="top",padx=10,pady=5)

#Gráfico 
#valores iniciales para el gráfico de barras:
Data1 = {'Clases': ['CARTON','CRISTAL','PLASTICO'], 'Porcentaje': [0,0,0]}
df1 = pd.DataFrame(Data1, columns= ['Clases', 'Porcentaje'])
df1 = df1[['Clases', 'Porcentaje']].groupby('Clases').sum()

#Crear Gráfico de barras:
grafico1 = plt.Figure(figsize=(6,6), dpi=70)
barras = grafico1.add_subplot(111)
bar1 = FigureCanvasTkAgg(grafico1, framegraficos)
bar1.get_tk_widget().pack()
#df1.xticks(rotation='horizontal')
df1.plot(kind='bar', legend=True, ax=barras)

barras.set_title('Porcentaje de coincidencia')

centro.pack(side=TOP,expand=True)

#------- Botones
label=Label(app,background='lavender', font=('arial',15,'bold'))

acconeer=Button(label,text="Iniciar lectura por radar",command=carga_acconeer,padx=10,pady=5)
upload=Button(label,text="Cargar archivo de datos",command=carga_datos,padx=10,pady=5)

acconeer.configure(background='#364156', foreground='white',font=('arial',10,'bold'))
upload.configure(background='#364156', foreground='white',font=('arial',10,'bold'))

upload.pack(side=BOTTOM, anchor=W, fill=X, expand=True)
acconeer.pack(side=BOTTOM, anchor=W, fill=X, expand=True)

label.pack(side=BOTTOM,fill=X,expand=True)


app.mainloop()